In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [19]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality',discretize=False, balance=True, dataset_folder=dataset_folder)

In [21]:
dataset.data[np.argwhere(dataset.labels==0)].shape

(4180, 1, 14)

In [3]:
explainer = lime.lime_tabular.LimeTabularExplainer(dataset.train, feature_names=dataset.feature_names, class_names=dataset.class_target,discretize_continuous=False,)

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(dataset.train, dataset.labels_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))

Train 1.0
Test 0.8504784688995215


In [5]:
preds =  predict_fn(dataset.test)

In [ ]:
import pickle
pickling_on = open("rfmort.pickle","rb")
c = pickle.load(pickling_on)
pickling_on.close()

In [5]:
import pickle
pickling_on = open("rfmort.pickle","wb")
pickle.dump(c, pickling_on)
pickling_on.close()

In [6]:
exp_fn = lambda i: explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in range(len(xtest)):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [7]:
exp1 = exp_fn_blk(dataset.test, exp_fn)

In [8]:
import copy
test_ori = []
enc_feats = []
sz = dataset.test.shape
for i in range(sz[0]):
    temp = list(copy.deepcopy(dataset.test[i]))
    for j in range(sz[0]):
        if j in dataset.categorical_features:
            temp[j] = dataset.categorical_names[j][int(temp[j])].replace(',','')
    test_ori.append(temp)

ori_data = pd.DataFrame(test_ori, columns=dataset.feature_names)

feats = copy.deepcopy(dataset.feature_names)

In [9]:
num_feats = len(dataset.feature_names)
sz = exp1.shape
imp = np.zeros(num_feats)
for i in range(sz[0]):
    for j in range(sz[1]):
        imp[int(exp1[i][j][0])] = imp[int(exp1[i][j][0])] + sz[1] - j
    

In [11]:
order = np.argsort(-imp)

In [12]:
a = list(np.array(feats)[order])

In [13]:
a

['age',
 'mets_achieved',
 'percent_hr_achieved',
 'sex',
 'smoke',
 'dm',
 'racewb',
 'resting_systolic',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'famhx',
 'afib2']

In [14]:
view1 = ori_data.reindex(columns=a)
viewno = copy.deepcopy(ori_data)

In [ ]:
view1

In [163]:
#view1.sort_values(by='age', inplace=True)

In [15]:
view1.insert(loc=0,column='dead',value=dataset.labels_test)
viewno.insert(loc=0,column='dead',value=dataset.labels_test)
view1.insert(loc=1,column='prediction',value=preds)
viewno.insert(loc=1,column='prediction',value=preds)

In [16]:
yesmap={0:'No', 1:'Yes'}
yes_no = lambda x: yesmap[x] 

In [17]:
view1['dead']=view1['dead'].apply(yes_no)
viewno['dead']=viewno['dead'].apply(yes_no)
view1['prediction']=view1['prediction'].apply(yes_no)
viewno['prediction']=viewno['prediction'].apply(yes_no)

In [18]:
view1.to_csv('amortlime.csv',index=False)
viewno.to_csv('amortno.csv',index=False)